# Perceptron implementation

This code demonstrates basic implementation of a perceptron using NumPy and Pandas packages.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functions import *

#print(help(generate_data))
#print(help(get_weighted_sum))
#print(help(sigmoid))

In [2]:
n_features = 3
m_examples = 15
data = generate_data(m_examples, n_features, seed=15)
print('Features and labels matrix:\n---------------------------\n')
print(data, '\n\n---------------------------')
X = data.drop(columns=['targets']).values
Y = data['targets'].values
print(f'Number of features: {X.shape[1]}')
print(f'Number of training examples: {X.shape[0]}')

Features and labels matrix:
---------------------------

          x0        x1        x2  targets
0   0.692743  0.815817  0.344407        1
1   0.044838  0.571597  0.146245        0
2   0.718771  0.345357  0.457010        1
3   0.975938  0.781466  0.843790        0
4   0.555114  0.941621  0.018326        1
5   0.889184  0.389758  0.231740        1
6   0.534394  0.944690  0.330998        1
7   0.908621  0.467332  0.967288        0
8   0.764227  0.783354  0.361527        0
9   0.587788  0.248549  0.800139        1
10  0.258527  0.672610  0.075345        1
11  0.246870  0.358120  0.172055        1
12  0.260295  0.201803  0.039764        0
13  0.342869  0.468250  0.417349        1
14  0.899524  0.088874  0.623966        1 

---------------------------
Number of features: 3
Number of training examples: 15


In [3]:
weights, bias = network_init(n_features, seed=18)
print('Initial network parameters:')
for i in range(len(weights)):
    print(f'w{i} = {weights[i]}')
print(f'b = {bias}')

Initial network parameters:
w0 = 0.39930576921757277
w1 = 0.7174146792882161
w2 = 0.2808233280360034
b = 0


# Forward propagation

In this section, forward propagation through the perceptron is performed 

In [4]:
def forward_prop(X, weights, bias):
    weighted_sums = []
    Y_hat = []
    for i in range(m_examples):
        example = X[i,:]
        weighted_sum = get_weighted_sum(example, weights, bias)
        weighted_sums.append(weighted_sum)
        y_hat = sigmoid(weighted_sum)
        Y_hat.append(y_hat)
    return Y_hat, weighted_sums

Y_hat, weighted_sums = forward_prop(X, weights, bias)
initial_pass = pd.DataFrame({'weighted_sum':weighted_sums, 'Y':Y, 'Y_hat':Y_hat})
print('Initial forward propagation results:\n---------------------------\n')
print(initial_pass)

Initial forward propagation results:
---------------------------

    weighted_sum  Y     Y_hat
0       0.958613  1  0.722844
1       0.469046  0  0.615158
2       0.663112  1  0.659959
3       1.187289  0  0.766256
4       0.902340  1  0.711430
5       0.699753  1  0.668133
6       0.984073  1  0.727916
7       0.969725  0  0.725065
8       0.968675  0  0.724855
9       0.637718  1  0.654237
10      0.606930  1  0.647240
11      0.403815  1  0.599604
12      0.259881  0  0.564607
13      0.590040  1  0.643374
14      0.598169  1  0.645237


## Cross-entropy loss

In [5]:
def cross_entropy(Y, Y_hat):
    cost = []
    for i in range(len(Y)):
        y = Y[i]
        y_hat = Y_hat[i]
        cross_entropy_loss = -(y*np.log10(y_hat) + (1-y)*np.log10(1-y_hat))
        cost.append(cross_entropy_loss)
    return cost
cross_entropy_cost = cross_entropy(Y, Y_hat)

initial_pass_with_cost = initial_pass.copy()
initial_pass_with_cost.drop(columns=['weighted_sum'], inplace=True)
initial_pass_with_cost['c-e loss'] = cross_entropy_cost
print(initial_pass_with_cost)

    Y     Y_hat  c-e loss
0   1  0.722844  0.140955
1   0  0.615158  0.414717
2   1  0.659959  0.180483
3   0  0.766256  0.631259
4   1  0.711430  0.147868
5   1  0.668133  0.175137
6   1  0.727916  0.137919
7   0  0.725065  0.560770
8   0  0.724855  0.560439
9   1  0.654237  0.184265
10  1  0.647240  0.188935
11  1  0.599604  0.222136
12  0  0.564607  0.361119
13  1  0.643374  0.191536
14  1  0.645237  0.190281
